# Customer Segmentation for a Sports Facility

<div class="alert alert-block alert-info">

[1. Objectives](#1st-bullet)<br>
[2. Import Data/Libraries](#2nd-bullet)<br>  

</div>

<div class="alert alert-block alert-success">

<a class="anchor" id="1st-bullet">    </a>
## 1. Objectives 
</div>

1. Explore the data and identify the variables that should be used to segment customers.
2. Identify customer segments
3. Justify the number of clusters chosen (taking in consideration the business use as well).
4. Explain the clusters found.
5. Suggest business applications for the findings and define general marketing approaches for each cluster.


<div class="alert alert-block alert-success">

<a class="anchor" id="2nd-bullet">    </a>
## 2. Import Libraries/Data
</div>

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
data = pd.read_csv('XYZ_sports_dataset.csv', sep =';') 